In [1]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
import ssl
ssl._create_default_https_context=ssl._create_unverified_context

In [3]:
# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

In [4]:
#constant
BATCH_SIZE = 128
NB_EPOCH = 40
NB_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()

In [5]:
#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 3s 0us/step
X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [6]:
# convert to categorical
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)
# float and normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [7]:
# network
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)       

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  if sys.path[0] == '':


In [8]:
# train
model.compile(loss='categorical_crossentropy', optimizer=OPTIM,
metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=BATCH_SIZE,
epochs=NB_EPOCH, validation_split=VALIDATION_SPLIT,
verbose=VERBOSE)
score = model.evaluate(X_test, Y_test,
batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

Train on 40000 samples, validate on 10000 samples
Epoch 1/40
40000/40000 [==============================] - 108s 3ms/step - loss: 1.7796 - accuracy: 0.3542 - val_loss: 1.4937 - val_accuracy: 0.4477
Epoch 2/40
40000/40000 [==============================] - 111s 3ms/step - loss: 1.3467 - accuracy: 0.5186 - val_loss: 1.1867 - val_accuracy: 0.5816
Epoch 3/40
40000/40000 [==============================] - 116s 3ms/step - loss: 1.1336 - accuracy: 0.5992 - val_loss: 0.9841 - val_accuracy: 0.6553
Epoch 4/40
40000/40000 [==============================] - 110s 3ms/step - loss: 0.9974 - accuracy: 0.6497 - val_loss: 0.8782 - val_accuracy: 0.6918
Epoch 5/40
40000/40000 [==============================] - 111s 3ms/step - loss: 0.9018 - accuracy: 0.6846 - val_loss: 1.0078 - val_accuracy: 0.6476
Epoch 6/40
40000/40000 [==============================] - 111s 3ms/step - loss: 0.8276 - accuracy: 0.7091 - val_loss: 0.8270 - val_accuracy: 0.7104
Epoch 7/40
40000/40000 [==============================] - 107s

In [9]:
#save model
model_json = model.to_json()
open('cifar10_architecture.json', 'w').write(model_json)
# And the weights learned by our deep network on the training set
model.save_weights('cifar10_weights.h5', overwrite=True)

In [19]:
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
import numpy as np
NUM_TO_AUGMENT=5
NB_EPOCH=50
#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
# convert to categorical
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)
# float and normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
# augumenting
print("Augmenting training set images...")
datagen = ImageDataGenerator(
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest')
xtas, ytas = [], []
for i in range(X_train.shape[0]):
    num_aug = 0
    x = X_train[i] # (3, 32, 32)
    x = x.reshape((1,) + x.shape) # (1, 3, 32, 32)   
for x_aug in datagen.flow(x, batch_size=1,
save_to_dir='preview', save_prefix='cifar', save_format='jpeg'):
    if num_aug >= NUM_TO_AUGMENT:
        break
    xtas.append(x_aug[0])
    num_aug += 1
#fit the dataget
datagen.fit(X_train)
# train
history = model.fit_generator(datagen.flow(X_train, Y_train,
batch_size=BATCH_SIZE), samples_per_epoch=X_train.shape[0],
epochs=NB_EPOCH, verbose=VERBOSE)
score = model.evaluate(X_test, Y_test,
batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Augmenting training set images...


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=50, verbose=1, steps_per_epoch=390)`


Epoch 1/50
390/390 [==============================] - 163s 417ms/step - loss: 2.3027 - accuracy: 0.0990
Epoch 2/50
390/390 [==============================] - 129s 330ms/step - loss: 2.3027 - accuracy: 0.0989
Epoch 3/50
390/390 [==============================] - 129s 331ms/step - loss: 2.3027 - accuracy: 0.1001
Epoch 4/50
390/390 [==============================] - 115s 294ms/step - loss: 2.3027 - accuracy: 0.0971
Epoch 5/50
390/390 [==============================] - 119s 304ms/step - loss: 2.3027 - accuracy: 0.0966
Epoch 6/50
390/390 [==============================] - 164s 420ms/step - loss: 2.3027 - accuracy: 0.0990
Epoch 7/50
390/390 [==============================] - 141s 362ms/step - loss: 2.3027 - accuracy: 0.0989
Epoch 8/50
390/390 [==============================] - 181s 464ms/step - loss: 2.3027 - accuracy: 0.0974
Epoch 9/50
390/390 [==============================] - 175s 449ms/step - loss: 2.3027 - accuracy: 0.0987
Epoch 10/50
390/390 [==============================] - 175s 448m

Today, the use of Convolutional Neural Networks (CNNs) is becoming more prevalent, and these deep-learning algorithms are<br> extraordinary at being able to identify complex patterns within images and videos, making them essential in today’s image<br> recognition tasks (Visionplatform.ai, 2024).<br><br>  
Since the way algorithms are trained is based on the data set used, these types of algorithms could certainly be used to<br> distinguish people’s faces (Visionplatform.ai, 2024). Once an algorithm is trained, and equipped with what it has learned from <br>the data set, it can analyze new images by breaking down each image into component elements, which are often pixels, where <br>the algorithm can then look for patterns and features it has been previously trained to recognize (Visionplatform.ai, 2024).<br><br>
Therefore, we can begin to better understand how we can feed any dataset to the algorithm, which can then be used to train it,<br> and then decide what we want to do with that trained algorithm, which includes facial recognition among a wide variety of <br>other uses of image recognition.<br><br>
Because it is possible to train algorithms for image recognition, such as facial recognition, it is significant to have an <br>understanding of its ethical and privacy implications.<br><br>
Ethical and privacy concerns that arise from facial recognition include, Racial bias due to testing inaccuracies, <br>discrimination in law enforcement, data privacy, lack of informed consent and transparency, mass surveillance, data <br>breaches, and lack of legal support (Gangarapu, 2022).<br><br>
As it concerns racial bias due to testing inaccuracies, it has been common that of the errors detected in facial recognition<br> systems used for law enforcement, errors were more common on dark-skinned faces, and it further showed fewer<br> errors when matching light-skinned faces (Gangarapu, 2022). Also concerning issues in law enforcement, it has been<br> found that facial recognition algorithms worked better for faces of middle-aged white males, but underperformed for <br>people of color (specifically 35% of errors occurred with women of color compared to 1% for white males), elderly people,<br> and women and children (Gangarapu, 2022). Overall, facial recognition could potentially result in penalizing individuals<br> for crimes they did not actually commit (Gangarapu, 2022).<br><br>
Privacy is a significant public concern regarding facial recognition because it “infringes on citizens’ inherent right to be<br> under constant government surveillance and keep their images without consent” (Gangarapu, 2022). This also means that<br> facial recognition can lead to mass surveillance, which compromises the liberty and privacy rights of citizens when<br> used alongside cameras that are everywhere and in conjunction with data analytics (Gangarapu, 2022).<br><br>
Also, when considering privacy, it is important to realize that many issues arise from unsecured data storage methods, which <br>expose this type of data and other security-related threats. Data breaches are of significant concern because they can lead<br> to facial identity in serious crimes; quite often leaving victims with very few legal remedies (Gangarapu, 2022). <br><br>
Lastly, the methods by which facial images are often collected are from publicly hosted sites, such as Facebook, where people<br> have openly shared their pictures, however, these images are found to be misused to help assess and advance <br>commercial surveillance products (Gangarapu, 2022). <br><br> 
References:<br><br>
Gangarapu, K. R. (2022, January 25). Ethics of Facial Recognition: Key Issues and Solutions. G2.com, Inc..<br> https://learn.g2.com/ethics-of-facial-recognition<br><br>
Visionplatform.ai. (2024, January 27). Understanding Image Recognition: Algorithms, Machine Learning, and Uses.<br> https://visionplatform.ai/image-recognition/
